<a href="https://colab.research.google.com/github/dr-song-summer-project/ChatBot_Preprocessing/blob/main/refining_Data/Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
# from nltk.tokenize import word_tokenize  
# print(word_tokenize("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."))  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git   #띄어쓰기
!pip install git+https://github.com/ssut/py-hanspell.git          #맞춤법
!pip install soynlp                  #반복 제거
!pip install kss                     #문장 구분

In [ ]:
#기초적 전처리, html tag 제거, 불필요 언어 제거, Lowercasing, punctuation(문장부호) 제거
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&' 
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', } 
def clean_punc(text, punct, mapping): 
  for p in mapping: 
    text = text.replace(p, mapping[p]) 

  for p in punct: 
    text = text.replace(p, f' {p} ') 
    
  specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''} 
  for s in specials: 
    text = text.replace(s, specials[s]) 
    
  return text.strip()

import re

def clean_text(texts):   
    corpus = []
   for i in range(0, len(texts)) : 
     review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation 
     review = re.sub(r'\d+','', str(texts[i]))# remove number 
     review = review.lower() #lower case 
     review = re.sub(r'\s+', ' ', review) #remove extra space 
     review = re.sub(r'<[^>]+>','',review) #remove Html tags 
     review = re.sub(r'\s+', ' ', review) #remove spaces 
     review = re.sub(r"^\s+", '', review) #remove space from start 
     review = re.sub(r'\s+$', '', review) #remove space from the end 
     corpus.append(review) 
   return corpus

IndentationError: ignored

In [ ]:
from google.colab import files
files.upload()


Saving 피임약_crawling_result.xlsx to 피임약_crawling_result.xlsx


{'피임약_crawling_result.xlsx': b'PK\x03\x04\x14\x00\x00\x00\x08\x00m\x8d\xe2R\x07AMb\x81\x00\x00\x00\xb1\x00\x00\x00\x10\x00\x00\x00docProps/app.xmlM\x8e=\x0b\x021\x10D\xff\xcaq\xbd\xb7A\xc1Bb@\xd0R\xb0\xb2\x0f{\x1b/\x90dC\xb2B~\xbe9\xc1\x8fn\x1eo\x18F\xdf\ng*\xe2\xa9\x0e-\x86T\x8f\xe3"\x92\x0f\x00\x15\x17\x8a\xb6N]\xa7n\x1c\x97h\xa5cy\x00;\xe7\x91\xce\x8c\xcfHI`\xab\xd4\x1e\xa8\t\xa5\x99\xe6M\xfe\x0e\x8eF\x9fr\x0e\x1e\xadxN\xe6\xea\xb1pe\'\xc3\xa5!\x05\r\xffrm\xde\xa9\xd45\xef&\xf5\x96\x1f\xd6\xf0;i^PK\x03\x04\x14\x00\x00\x00\x08\x00m\x8d\xe2R[\xfa\xdd\xba\xee\x00\x00\x00+\x02\x00\x00\x11\x00\x00\x00docProps/core.xml\xcd\x92\xc1j\xc30\x0c\x86_e\xf8\x9e\xc8N\xe8\x06&\xf5\xa5\xa3\xa7\x0e\x06+l\xecfl\xb55\x8b\x13ck$}\xfb9Y\x9b2\xb6\x07\xd8\xd1\xd2\xefO\x9f@\x8d\t\xd2\xf4\x11\x9fc\x1f0\x92\xc3t7\xfa\xb6K\xd2\x845;\x11\x05\t\x90\xcc\t\xbdNeNt\xb9y\xe8\xa3\xd7\x94\x9f\xf1\x08A\x9b\x0f}D\xa88\xbf\x07\x8f\xa4\xad&\r\x13\xb0\x08\x0b\x91\xa9\xc6\x1ai"j\xea\xe3\x05o\xcd\x82\x0f\x9f\xb1\x9da\xd6\x0

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import nltk
import pandas as pd
from openpyxl import Workbook
from pykospacing import Spacing 
from hanspell import spell_checker
from tqdm.notebook import tqdm
from soynlp.normalizer import *
import kss
import re

# ============================================
# --  텍스트 전처리
# ============================================
spacing = Spacing()
def spell_text(texts):
  sample_space = spacing(texts)
  # spelled_sent = spell_checker.check(sample_space)
  # sampel_space_spell = spelled_sent.checked
  return (sample_space)

def process_sentence(texts):
  tmp = ''
  for sent_ in kss.split_sentences(texts):
    if(('안녕하세요' in sent_) or ('하이닥' in sent_) or ('감사합니다' in sent_) or (len(sent_))>500) :
      continue
    tmp = tmp + ' ' + (spell_text(sent_))
    #print(spell_text(sent_), end=' ')  
  return tmp

# ============================================
# --  파일 읽어오기
# ============================================
df = pd.read_excel('/content/피임약_crawling_result.xlsx')
df_np = pd.DataFrame.to_numpy(df)

processed_sentence_question = []
processed_sentence_answer = []

for i in tqdm(range(0, len(df))):
  processed_sentence_question.append(process_sentence(df_np[i][1]))
  processed_sentence_answer.append(process_sentence(df_np[i][2]))  
  #print(str(i)+' = '+processed_sentence_answer[i])

# ============================================
# --  Excel 입력
# ============================================
wb = Workbook() 
sheet = wb.active 
sheet.title = '전처리된 문장' # 컬럼명 지정(헤더) 

sheet.cell(row=1, column=1, value='질문')
sheet.cell(row=1, column=2, value='답변')


# 시트 저장 
row_no = 2
for n, rows in tqdm(enumerate(processed_sentence_question)): 
  for seq, value in enumerate(rows): 
    sheet.cell(row=row_no+n, column=1, value=processed_sentence_question[n]) 
    sheet.cell(row=row_no+n, column=2, value=processed_sentence_answer[n])

wb.save('/content/sample_data_processed.xlsx')
wb.close()

